# Installing and Importing Necessary Packages

In [ ]:
import os
from os import walk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from tqdm import tqdm

In [ ]:
try:
    from vit_pytorch.t2t import T2TViT
except:
    !pip install vit_pytorch
    from vit_pytorch.t2t import T2TViT

import torch
from torchvision import models

try:
    from torchsummary import summary
except:
    !pip install torchsummary
    from torchsummary import summary


In [ ]:

from torchvision import models
from torchsummary import summary
#summary(model_resnet, (3, 224, 224))

In [ ]:
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn import Linear
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torch.nn import Conv2d
from torch.nn import BatchNorm2d
from torch.nn import MaxPool2d
from torch.nn import Dropout
from torch.nn import Sequential
import torchvision.models as models



In [ ]:
try:
    import torchbearer
except:
    !pip install -q torchbearer
    import torchbearer

In [ ]:
try:
    from efficientnet_pytorch import EfficientNet
except:
    !pip install efficientnet_pytorch
    from efficientnet_pytorch import EfficientNet

# Preparing the trainer

In [ ]:
import torchbearer
from torchbearer.callbacks import imaging

inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
    std=[1/0.229, 1/0.224, 1/0.255]
)

make_grid = imaging.MakeGrid(torchbearer.INPUT, num_images=64, nrow=8, transform=inv_normalize)
make_grid = make_grid.on_test().to_pyplot().to_file('sample.png')

In [ ]:
import torchbearer
from torchbearer import Trial

# If you have tensorboardX installed then write to tensorboard, else don't
import sys
if 'tensorboardX' in sys.modules:
  import tensorboardX
  from torchbearer.callbacks import TensorBoard
  callbacks = [TensorBoard(write_batch_metrics=True)]
else:
  callbacks = []

callbacks.append(make_grid)

# Connecting with Google drive and importing Dataset from kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Arranging the Dataset and Dataloader

In [ ]:
HP = {
    'epochs': 25,
    'batch_size': 32,
    'learning_rate': 1e-3,
    'momentum': 0.9,
    'test_size': 0.05,
    'seed': 1
}

In [ ]:
torch.manual_seed(HP['seed'])
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.benchmark = True
print(f'using {device} device')

using cuda device


In [ ]:
import pandas as pd
dataset_dir = 'train_images'
submission_dir = 'test_images'
submission_sample = 'sample_submission.csv'
submission_output = 'submission.csv'

dataset_file = '/content/gdrive/MyDrive/CSV file of rice disease/Ricedisease_shuffled2.csv'


df = pd.read_csv(dataset_file)


print(f'count: {len(df)} \n')
df.head(5)

count: 14685 



,Unnamed: 0,Image Location,Category No,Category
0,11733,/content/gdrive/MyDrive/Paddy big dataset/trai...,8,bacterial_leaf_blight
1,1231,/content/gdrive/MyDrive/Paddy big dataset/trai...,0,dead_heart
2,6876,/content/gdrive/MyDrive/Paddy big dataset/trai...,5,hispa
3,1768,/content/gdrive/MyDrive/Paddy big dataset/trai...,1,downy_mildew
4,2474,/content/gdrive/MyDrive/Paddy big dataset/trai...,2,normal


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:

train_df, test_df = train_test_split(df, test_size=HP['test_size'])
print(f'train len: {len(train_df)}, test len: {len(test_df)}')

train len: 13950, test len: 735


In [ ]:
train_df['Category'].value_counts()

blast                       2525
brown_spot                  2443
bacterial_leaf_blight       1722
normal                      1679
hispa                       1521
tungro                      1417
dead_heart                  1363
downy_mildew                 595
bacterial_leaf_streak        362
bacterial_panicle_blight     323
Name: Category, dtype: int64

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomChoice([
        transforms.Pad(padding=10),
        transforms.CenterCrop(480),
        transforms.RandomRotation(60),
        transforms.CenterCrop((576,432)),
        transforms.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.1
        )
    ]),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class PaddyDataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df

        self.transforms = transforms
        #self.df = self.df.values.tolist()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        image = Image.open(self.df.iloc[idx]['Image Location']).convert('RGB')
        image = self.transforms(image)
        idx =self.df.iloc[idx]['Category No']

        return image, int(idx)


train_dataset = PaddyDataset(train_df, train_transform)
test_dataset = PaddyDataset(test_df, test_transform)
#test_dataloader = DataLoader(test_dataset, batch_size=HP['batch_size'], shuffle=True, pin_memory=True)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=55, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=55, shuffle=True, pin_memory=True)

In [ ]:
#print(train_dataset.__getitem__(10)[0].shape)

In [ ]:
#print(train_dataset.__getitem__(11)[0].shape)

# Trainer

In [ ]:
def trainer(model,epochs,lr,device=device):
  loss = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  trial = Trial(model, optimizer,loss, metrics=['acc', 'loss'], callbacks=callbacks).to(device)
  trial.with_generators(train_generator=train_dataloader, val_generator=test_dataloader)
  history = trial.run(epochs=epochs, verbose=2)
  return history

# Save Model

In [ ]:
import os
def save_model(model,savefile_Name):
  checkpoint = {'state_dict': model.state_dict()}
  torch.save(checkpoint, os.path.join('gdrive/MyDrive/',savefile_Name))

# Model

In [ ]:
!pip install torch-intermediate-layer-getter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torch_intermediate_layer_getter import IntermediateLayerGetter as MidGetter

Efficientnet-B2+2layer VIT:

In [ ]:
class EfficientnetB2_VIT(Module):
  #formula of output dimension: (𝑛 + 2𝑝 − 𝑓) / 𝑠} + 1
  def __init__(self):
    super().__init__()

    self.model =EfficientNet.from_pretrained('efficientnet-b2',num_classes=10)
    self.eff_to_vit_conv=Conv2d(1408,550,1)


    return_layers = {
    '_bn1': '_bn1',

    }
    self.mid_getter = MidGetter(self.model, return_layers=return_layers, keep_output=True)

    self.vit = T2TViT(
    dim =128,
    image_size =7,
    channels=550,
    depth = 2,
    heads = 8,
    mlp_dim=64,
    num_classes = 10,
    t2t_layers = ((1, 1), (1, 1), (1, 1)) # tuples of the kernel size and stride of each consecutive layers of the initial token to token module
    )

    self.threshold=0.3
    self.softmax=Softmax(dim=1)
    self.relu=ReLU()
  def forward(self,x):
    vit_input, efficientnet_output = self.mid_getter(x)
    #print(vit_input)
    vit_input=self.eff_to_vit_conv(vit_input['_bn1'])
    vit_output=self.vit(vit_input)
    out=efficientnet_output*self.threshold+vit_output*(1-self.threshold)
    return out


In [ ]:
efficientnetb2_vit= EfficientnetB2_VIT()
efficientnetb2_vit.threshold=0.33

Loaded pretrained weights for efficientnet-b2


In [ ]:
efficientnetb2_vit.load_state_dict(torch.load('/content/gdrive/My Drive/Paddy Disease Classification/efficientnetb2_vit_0.33.pth')['state_dict'])

<All keys matched successfully>

In [ ]:
summary(efficientnetb2_vit.to('cuda'),(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 225, 225]               0
Conv2dStaticSamePadding-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
MemoryEfficientSwish-4         [-1, 32, 112, 112]               0
         ZeroPad2d-5         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 32, 112, 112]             288
       BatchNorm2d-7         [-1, 32, 112, 112]              64
MemoryEfficientSwish-8         [-1, 32, 112, 112]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:

#Uncomment if you want to
#efficientnetb2_vit.to('cuda')
efficientnetb2_vit.train()
save=torchbearer.callbacks.checkpointers.Best(filepath='efficientnetb2_vit_0.33_2.pth', monitor='val_loss', mode='auto')
#save=save_model(efficientnetb2_vit,'efficientnetb2_vit_0.33_2.pth')
callbacks.append(save)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(efficientnetb2_vit.parameters(), lr=3e-8)
trial = Trial(efficientnetb2_vit, optimizer,loss, metrics=['acc', 'loss'], callbacks=callbacks).to(device)
trial.with_generators(train_generator=train_dataloader, val_generator=test_dataloader)
history = trial.run(epochs=2, verbose=2)
#history=trainer(efficientnetb2_vit,6,5e-4)

0/2(t):   0%|          | 0/254 [00:00<?, ?it/s]

0/2(v):   0%|          | 0/14 [00:00<?, ?it/s]

1/2(t):   0%|          | 0/254 [00:00<?, ?it/s]

1/2(v):   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:

#Uncomment if you want to save the model

save_model(efficientnetb2_vit,'efficientnetb2_vit_0.33_2.pth')

# Cuda free memory

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)


# Kaggle Submission File

Competition Link:https://www.kaggle.com/competitions/paddy-disease-classification

This Submission scores 98.332% accuracy on test dataset of this competition

In [ ]:
%%time

import functools
import gc

efficientnetb2_vit.to(device)
efficientnetb2_vit.eval()

image_ids, labels = [], []
i=1



for (dirpath, dirname, filenames) in walk(submission_dir):
    for filename in filenames:
        image = Image.open('test_images'+'/'+filename)
        image = test_transform(image)
        image = image.unsqueeze(0).to(device)
        print("Image_no(Out of 3469 images):")
        print(i,end="\r")
        i+=1

        image_ids.append(filename)
        labels.append(idx_to_label[tensor_val.argmax().item()])

        image.to('cpu')
        del(image)
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:

submission = pd.DataFrame({
    'image_id': image_ids,
    'label': labels,
})


# submission['label'].value_counts()

In [ ]:
submission.to_csv('pred.csv')

In [ ]:
submission.to_csv(submission_output, index=False, header=True)